In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import utils
from datetime import datetime


In [ ]:
import boto3

session = boto3.Session(profile_name='AdministratorAccess', region_name='us-east-2')
# resource vs client: https://www.learnaws.org/2021/02/24/boto3-resource-client/
dynamodb_resource = session.resource('dynamodb')  #  higher level abstractions, recommended to use, fewer methods but creating table returns a table object that you can run operations on, can also grab a Table with Table('name')
# dynamodb_client = session.client('dynamodb')  # low-level, more explicit methods. Creating table returns a dictionary

# Get Rising Data


In [ ]:
risingTable = dynamodb_resource.Table('rising')

In [ ]:
datesToQuery = utils.daysUntilNow()
print("Dates to query:", datesToQuery)

In [ ]:
postIdQueryResult = utils.queryByRangeOfDates(risingTable, datesToQuery)  # [{'postId': XXXXXX}, {'postId': YYYYYY}...]
postsOfInterest = {res['postId'] for res in postIdQueryResult}

In [ ]:
print("Number of posts found:", len(postsOfInterest))

In [ ]:
from pyspark.sql import SparkSession

cfg_file = utils.findConfig()
cfg = utils.parseConfig(cfg_file)

spark = (
  SparkSession
  .builder
  .appName('redditData')
  .config("fs.s3a.access.key", cfg['ACCESSKEY'])
  .config("fs.s3a.secret.key", cfg['SECRETKEY'])
  .getOrCreate()
)


In [ ]:
# this can take a while due to read constraints placed on dynamo db, consider increasing RCU on database
# it can also be slow because converts each dynamodb partition to a spark dataframe,
# this was done so that it would scale better on a distributed system 
# over keeping all the data in python in one node and trying to then move it to spark
postIdData = utils.getPostIdSparkDataFrame(spark, risingTable, postsOfInterest)

In [ ]:
pandasTestDf = postIdData.limit(5).toPandas()
pandasTestDf.head()

In [ ]:
import pyspark.sql.functions as F

aggData = (
  postIdData
  .groupBy('postId', 'subreddit', 'title', 'createdTSUTC')
  .agg(
    F.max(F.when(F.col('timeElapsedMin') <= 20, F.col('score'))).alias('maxScore20m')
    , F.max(F.when(F.col('timeElapsedMin').between(21,40), F.col('score'))).alias('maxScore21_40m')
    , F.max(F.when(F.col('timeElapsedMin').between(41,60), F.col('score'))).alias('maxScore41_60m')
    , F.max(F.when(F.col('timeElapsedMin') <= 20, F.col('numComments'))).alias('maxNumComments20m')
    , F.max(F.when(F.col('timeElapsedMin').between(21,40), F.col('numComments'))).alias('maxNumComments21_40m')
    , F.max(F.when(F.col('timeElapsedMin').between(41,60), F.col('numComments'))).alias('maxNumComments41_60m')
    , F.max(F.when(F.col('timeElapsedMin') <= 20, F.col('upvoteRatio'))).alias('maxUpvoteRatio20m')
    , F.max(F.when(F.col('timeElapsedMin').between(21,40), F.col('upvoteRatio'))).alias('maxUpvoteRatio21_40m')
    , F.max(F.when(F.col('timeElapsedMin').between(41,60), F.col('upvoteRatio'))).alias('maxUpvoteRatio41_60m')
    , F.max(F.when(F.col('timeElapsedMin') <= 20, F.col('numGildings'))).alias('maxNumGildings20m')
    , F.max(F.when(F.col('timeElapsedMin').between(21,40), F.col('numGildings'))).alias('maxNumGildings21_40m')
    , F.max(F.when(F.col('timeElapsedMin').between(41,60), F.col('numGildings'))).alias('maxNumGildings41_60m')
  )
)

# Get Targets

In [ ]:
hotTable = dynamodb_resource.Table('hot')

In [ ]:
hotPosts = utils.queryByRangeOfDates(hotTable, datesToQuery)
uniqueHotPostIds = set([p['postId'] for p in hotPosts])

In [ ]:
# the hot posts are usually not a very long list and we really only need this for the purpose of creating targets
print("unique hot postIds:", uniqueHotPostIds)

In [ ]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf

def getTarget(postId:str, uniqueHotPostIds:set):
  if postId in uniqueHotPostIds:
    return 1
  else:
    return 0
  
getTargetUDF = udf(lambda x: getTarget(x, uniqueHotPostIds), returnType=IntegerType())


aggData = aggData.withColumn('target', getTargetUDF(F.col('postId')))

In [ ]:
# by aggregating the data, there should be an at most 60x reduction in the data (since data can be collected once every minute)

aggDataPd = aggData.toPandas()

In [ ]:
len(aggDataPd)

In [ ]:
aggDataPd.head()

In [ ]:
aggDataPd[aggDataPd['target']==1]

In [ ]:
for pId in aggDataPd[aggDataPd['target']==1]['postId']:
  print('https://reddit.com/'+pId)

At the time of writing, I've only collected about 1.5 days of data, with 7 viral posts (not a very large amount although that was to be expected). Interestingly, I've noticed that of the viral posts I have, 

- the one that had the most upvotes after an hour was actually the least viral, 
- while the one with the least upvotes was actually the most viral.
- but that post with the least upvotes had the second most comments of the viral posts, 24 comments, so maybe it would be captured by the model

I'm considering extending the time out to 90-120 minutes for data collection. However, the point was to get to a post early when there were relatively few comments. That most viral post had 24 comments after an hour and even that is a lot and any new replies are likely to be buried.




# Write to S3

This is basically our model data and what we will use to train a model. I used spark to write to s3 to future proof this if the data was too large to fit in pandas on driver.

If you get an error here then you probably need to download hadoop-aws-*.jar (ex: [3.2.0](https://mvnrepository.com/artifact/org.apache.hadoop/hadoop-aws/3.2.0)) and aws-java-sdk-bundle-*.jar (ex: [1.11.375](https://mvnrepository.com/artifact/com.amazonaws/aws-java-sdk-bundle/1.11.375))

- for hadoop-aws-*.jar this should match the version of other hadoop jars in $SPARK_HOME/jars/
- for aws-java-sdk-bundle-*.jar you will need to check the version dependency of hadoop-aws-*.jar on the maven website. Do NOT use the upgraded version, use the version that hadoop-aws was created with.

You may not need to add these dependencies to the the configs, but you may need to restart the kernel and rerun things.

These links may help:

- [SO link 1](https://stackoverflow.com/questions/58415928/spark-s3-error-java-lang-classnotfoundexception-class-org-apache-hadoop-f?answertab=scoredesc#tab-top)
- [SO link 2](https://stackoverflow.com/questions/44411493/java-lang-noclassdeffounderror-org-apache-hadoop-fs-storagestatistics/44500698#44500698)
- [SO link 3](https://stackoverflow.com/questions/64547468/pyspark-s3-error-java-lang-noclassdeffounderror-com-amazonaws-amazonserviceex)
- [Tutorial](https://notadatascientist.com/running-apache-spark-and-s3-locally/)
- [Hadoop Troubleshooting](https://hadoop.apache.org/docs/stable/hadoop-aws/tools/hadoop-aws/troubleshooting_s3a.html)

In [ ]:
aggData.write.parquet("s3a://data-kennethmyers/redditAggregatedData.parquet", mode="overwrite")
